In [5]:
import requests
import csv
import io
import sqlite3


In [6]:
import csv
import sqlite3
import datetime

# Create a connection to the SQLite database
db_conn = sqlite3.connect('debt_collection.db')
cursor = db_conn.cursor()

# Create the borrowers table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS borrowers (
        id INTEGER PRIMARY KEY,
        name VARCHAR(50),
        date_of_birth DATE,
        gender VARCHAR(10),
        marital_status VARCHAR(20),
        phone_number VARCHAR(20),
        email_address VARCHAR(50),
        mailing_address VARCHAR(100),
        language_preference VARCHAR(20),
        geographical_location VARCHAR(50),
        credit_score INTEGER,
        loan_type VARCHAR(20),
        loan_amount DECIMAL(10, 2),
        loan_term INTEGER,
        interest_rate DECIMAL(5, 2),
        loan_purpose VARCHAR(50),
        emi DECIMAL(10, 2),
        ip_address VARCHAR(20),
        geolocation VARCHAR(50),
        days_left_to_pay_current_emi INTEGER,
        delayed_payment VARCHAR(10)
    )
''')

# Create the repayment_history table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS repayment_history (
        id INTEGER PRIMARY KEY,
        borrower_id INTEGER,
        payment_date DATE,
        payment_mode VARCHAR(20),
        FOREIGN KEY (borrower_id) REFERENCES borrowers (id)
    )
''')

# Insert data into the borrowers table
with open('5k_borrowers_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        # Convert date of birth to datetime object
        dob = datetime.datetime.strptime(row[1], '%d-%m-%Y').date()

        # Insert data into the borrowers table
        cursor.execute('''
            INSERT INTO borrowers (
                name, date_of_birth, gender, marital_status, phone_number, email_address,
                mailing_address, language_preference, geographical_location, credit_score,
                loan_type, loan_amount, loan_term, interest_rate, loan_purpose, emi,
                ip_address, geolocation, days_left_to_pay_current_emi, delayed_payment
            ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''', (
            row[0], dob, row[2], row[3], row[4], row[5], row[6], row[7], row[8],
            int(row[9]), row[10], float(row[11]), int(row[12]), float(row[13]), row[14],
            float(row[15]), row[16], row[17], int(row[19]), row[20]
        ))

        # Get the last inserted row's ID
        borrower_id = cursor.lastrowid

        # Parse the repayment history string
        repayment_history_str = row[18]
        repayment_history = eval(repayment_history_str)

        # Insert data into the repayment_history table
        for payment in repayment_history:
            payment_date = payment['Payment Date']
            payment_mode = payment['Payment Mode']
            cursor.execute('''
                INSERT INTO repayment_history (
                    borrower_id, payment_date, payment_mode
                ) VALUES (?,?,?)
            ''', (borrower_id, payment_date, payment_mode))

# Commit the changes and close the database connection
db_conn.commit()
db_conn.close()